In [3]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
import seaborn as sbn
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn import preprocessing

### Question 1. Fit a polynomial model 
of degree $M=10$ to the data below. Perform Lasso regularization, fitting the model over data1_1, selecting the optimal value of the regularization parameter over the data1_2 (based on validation R2) and testing performance over the data1_3.

Important - standardize the data before training Lasso model and apply the same $\mu$ and $\sigma$ defined over data1_1 to data1_2 and data1_3 during validation/testing (as we are not supposed to learn anything, including normalization coefficients from validation and test data; all the parameters of the model are to be learned from training data exclusively).

Visualize the final model against all the data from from training, validation and test samples on the same plot using different colors for points from different samples. For comparison also visualize the true model used to generate the data below - $y=x^4/100+x^3/20+x^2/3+2$.

Output the coefficients of the model.

In [39]:
#generate data and put it in the dataframe
np.random.seed(2018)
x=np.arange(-10,20,0.5)
y=x**4/100+x**3/20+x**2/3+2*x+np.random.normal(loc=0,scale=3,size=60)
data1=pd.DataFrame({'x':x,'y':y}) #create a dataframe
#slice the data in three pieces (we'll talk about those later)
data1_2=data1.loc[40:49]
data1_3=data1.loc[50:59]
data1=data1.loc[0:39] 
data1.head() #for now let's stick with this first one

,x,y
0,-10.0,62.503031
1,-9.5,51.410761
2,-9.0,44.605198
3,-8.5,24.739247
4,-8.0,22.200164


In [40]:
#generate regressors for data1, data1_2, data1_3
def square(list):
    return [i ** 2 for i in list]
def triple(list):
    return [i ** 3 for i in list]
x2 = square(data1['x'])
x3 = triple(data1['x'])
x4 = square(x2)
x5 = triple(x2)
x6 = square(x4)
x7 = triple(x4)
x8 = square(x6)
x9 = triple(x6)
x10 = square(x8)
data1 = pd.DataFrame(data=[list(data1['x']),list(data1['y']),x2,x3,x4,x5,x6,x7,x8,x9,x10], index=(['x', 'y', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10'])).T
data1.head()

,x,y,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,-10.0,62.503031,100.00,-1000.000,10000.0000,1000000.000000,1.000000e+08,1.000000e+12,1.000000e+16,1.000000e+24,1.000000e+32
1,-9.5,51.410761,90.25,-857.375,8145.0625,735091.890625,6.634204e+07,5.403601e+11,4.401267e+15,2.919890e+23,1.937115e+31
2,-9.0,44.605198,81.00,-729.000,6561.0000,531441.000000,4.304672e+07,2.824295e+11,1.853020e+15,7.976644e+22,3.433684e+30
3,-8.5,24.739247,72.25,-614.125,5220.0625,377149.515625,2.724905e+07,1.422418e+11,7.425109e+14,2.023272e+22,5.513224e+29
4,-8.0,22.200164,64.00,-512.000,4096.0000,262144.000000,1.677722e+07,6.871948e+10,2.814750e+14,4.722366e+21,7.922816e+28


In [41]:
x2_2 = square(data1_2['x'])
x2_3 = triple(data1_2['x'])
x2_4 = square(x2_2)
x2_5 = triple(x2_2)
x2_6 = square(x2_4)
x2_7 = triple(x2_4)
x2_8 = square(x2_6)
x2_9 = triple(x2_6)
x2_10 = square(x2_8)
data1_2 = pd.DataFrame(data=[list(data1_2['x']),list(data1_2['y']),x2_2,x2_3,x2_4,x2_5,x2_6,x2_7,x2_8,x2_9,x2_10], index=(['x', 'y', 'x2_2', 'x2_3', 'x2_4', 'x2_5', 'x2_6', 'x2_7', 'x2_8', 'x2_9', 'x2_10'])).T
data1_2=data1_2.dropna()
data1_2

,x,y,x2_2,x2_3,x2_4,x2_5,x2_6,x2_7,x2_8,x2_9,x2_10
0,10.0,204.579091,100.00,1000.000,10000.0000,1.000000e+06,1.000000e+08,1.000000e+12,1.000000e+16,1.000000e+24,1.000000e+32
1,10.5,234.807204,110.25,1157.625,12155.0625,1.340096e+06,1.477455e+08,1.795856e+12,2.182875e+16,3.225100e+24,4.764941e+32
2,11.0,272.883209,121.00,1331.000,14641.0000,1.771561e+06,2.143589e+08,3.138428e+12,4.594973e+16,9.849733e+24,2.111378e+33
3,11.5,318.039709,132.25,1520.875,17490.0625,2.313061e+06,3.059023e+08,5.350250e+12,9.357621e+16,2.862518e+25,8.756507e+33
4,12.0,368.274565,144.00,1728.000,20736.0000,2.985984e+06,4.299817e+08,8.916100e+12,1.848843e+17,7.949685e+25,3.418219e+34
5,12.5,419.021459,156.25,1953.125,24414.0625,3.814697e+06,5.960464e+08,1.455192e+13,3.552714e+17,2.117582e+26,1.262177e+35
6,13.0,471.129773,169.00,2197.000,28561.0000,4.826809e+06,8.157307e+08,2.329809e+13,6.654166e+17,5.428008e+26,4.427793e+35
7,13.5,544.265746,182.25,2460.375,33215.0625,6.053445e+06,1.103240e+09,3.664420e+13,1.217139e+18,1.342797e+27,1.481428e+36
8,14.0,614.653032,196.00,2744.000,38416.0000,7.529536e+06,1.475789e+09,5.669391e+13,2.177953e+18,3.214200e+27,4.743481e+36
9,14.5,695.926336,210.25,3048.625,44205.0625,9.294114e+06,1.954088e+09,8.638056e+13,3.818458e+18,7.461602e+27,1.458062e+37


In [42]:
x3_2 = square(data1_3['x'])
x3_3 = triple(data1_3['x'])
x3_4 = square(x3_2)
x3_5 = triple(x3_2)
x3_6 = square(x3_4)
x3_7 = triple(x3_4)
x3_8 = square(x3_6)
x3_9 = triple(x3_6)
x3_10 = square(x3_8)
data1_3 = pd.DataFrame(data=[list(data1_3['x']),list(data1_3['y']),x3_2,x3_3,x3_4,x3_5,x3_6,x3_7,x3_8,x3_9,x3_10], index=(['x', 'y', 'x3_2', 'x3_3', 'x3_4', 'x3_5', 'x3_6', 'x3_7', 'x3_8', 'x3_9', 'x3_10'])).T
data1_3

,x,y,x3_2,x3_3,x3_4,x3_5,x3_6,x3_7,x3_8,x3_9,x3_10
0,15.0,780.565002,225.00,3375.000,50625.0000,1.139062e+07,2.562891e+09,1.297463e+14,6.568408e+18,1.683411e+28,4.314399e+37
1,15.5,869.737016,240.25,3723.875,57720.0625,1.386725e+07,3.331606e+09,1.923005e+14,1.109960e+19,3.697948e+28,1.232010e+38
2,16.0,976.252678,256.00,4096.000,65536.0000,1.677722e+07,4.294967e+09,2.814750e+14,1.844674e+19,7.922816e+28,3.402824e+38
3,16.5,1086.327787,272.25,4492.125,74120.0625,2.017919e+07,5.493784e+09,4.071996e+14,3.018166e+19,1.658115e+29,9.109325e+38
4,17.0,1208.637828,289.00,4913.000,83521.0000,2.413757e+07,6.975757e+09,5.826222e+14,4.866119e+19,3.394487e+29,2.367912e+39
5,17.5,1345.296972,306.25,5359.375,93789.0625,2.872290e+07,8.796388e+09,8.250050e+14,7.737645e+19,6.806333e+29,5.987114e+39
6,18.0,1483.542683,324.00,5832.000,104976.0000,3.401222e+07,1.101996e+10,1.156831e+15,1.214395e+20,1.338259e+30,1.474756e+40
7,18.5,1638.569608,342.25,6331.625,117135.0625,4.008948e+07,1.372062e+10,1.607166e+15,1.882555e+20,2.582983e+30,3.544013e+40
8,19.0,1804.894611,361.00,6859.000,130321.0000,4.704588e+07,1.698356e+10,2.213315e+15,2.884414e+20,4.898763e+30,8.319845e+40
9,19.5,1979.610105,380.25,7414.875,144590.0625,5.498037e+07,2.090629e+10,3.022841e+15,4.370728e+20,9.137569e+30,1.910326e+41


In [43]:
#consider just the model of degree M=10

In [3]:
#fit Lasso for various alpha and tune it to optimize Validation R2

In [6]:
#report test R2

In [5]:
#visualize the model against all the data from training, validation and test samples 

### Question 2. P-values and hypothesis testing
Suppose that a multiple regression with 7 regressors gave the following p-values for each of them:
0.02, 0.1, 3e-15, 0.04, 0.001, 0.06, 0.03.

For how many regressors you would reject the null-hypothesis that their corresponding regression coefficient is zero at the 95% confidence level? Please explain.

### For five regressors I would reject the null-hypothesis that their corresponding regression coefficient is zero at the 95%. They are 0.02, 3e-15, 0.04, 0.001 and 0.03. Because they are all smaller than 0.05

### Question 3. Confidence intervals
Assume we perform a regression and get an estimate 10 for the slope coefficient for the regressor of interest. Select all statements that can not be true for its confidence intervals:

a. 95%-confidence interval is [-5, 5]

b. 99%-conficence interval is [9.99,10.01]

c. 95%-confidence interval is [9,100]

d. 99%-confidence interval is [8, 12], while 95%-confidence interval is [9, 11].

Explain your choices

### The answer are A&C. Because the mean is suppose to be 10. A and C's mean are clearly not 10.

# Principal component regression

### Question 4

Using the median price per sq.foot from Zillow data below as reported for 2018-08, fit the linear model using N leading principal components of the 311 data and perform cross-validation. Visualize performance depending on the number N of leading PCs. Specifically: 
    1. implement a 20-times-cross-validation as a function of N returing OS R2
    2. run it for N=1,2,...30, return the best N and the corresponding R2
    3. mark it with a vertical line on the plot and put the value of N and the corresponding R2 as text labels
**Important:** keep training pca over the training sample only, applying the same pca transform to the test samples in order to prepare regressors for them

In [37]:
# Write code here
zillow=pd.read_csv("Zip_MedianListingPricePerSqft_AllHomes.csv",index_col=0)
zillow.head()

,City,State,Metro,CountyName,SizeRank,2010-01,2010-02,2010-03,2010-04,2010-05,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08
RegionName,,,,,,,,,,,,,,,,,,,,,
10023,New York,NY,"New York, NY",New York,1,NaN,NaN,NaN,NaN,1366.621067,...,1954.992968,1983.055556,1963.986600,1702.573836,1522.948540,1568.100358,1612.378790,1569.371728,1599.538839,1619.794484
60614,Chicago,IL,"Chicago, IL",Cook,2,347.391304,333.975410,332.666667,324.490763,325.832510,...,514.255544,522.193211,519.321330,520.408163,476.592732,463.928571,470.769231,461.960986,455.861397,436.111111
79936,El Paso,TX,"El Paso, TX",El Paso,3,88.688320,88.159032,87.940589,87.929656,87.368706,...,88.419016,88.540363,88.755279,88.455330,88.367347,89.118199,90.128755,90.316333,90.726125,91.668726
10002,New York,NY,"New York, NY",New York,4,NaN,NaN,NaN,NaN,NaN,...,2041.247701,2043.165468,2063.785322,2012.302285,2015.369804,2026.353276,2022.274326,2013.831259,1997.942387,2030.259366
926,San Juan,TX,"Brownsville, TX",Cameron,5,NaN,NaN,NaN,NaN,NaN,...,92.968750,92.954963,93.776641,92.968750,95.090118,103.846154,100.411523,108.447489,102.270729,102.270729


**Load/process 311 data**

In [46]:
data311 = pd.read_csv( 'aggr311.csv' , index_col=0 )
data311.head()

,Zip,Complain,Count
1,NaN,Adopt-A-Basket,5
2,10001,Adopt-A-Basket,1
3,10003,Adopt-A-Basket,1
4,10009,Adopt-A-Basket,1
5,10010,Adopt-A-Basket,1


In [47]:
data311.Zip=pd.to_numeric(data311.Zip,errors='coerce')
data311=data311.loc[(data311.Zip>=10000)&(data311.Zip<11500)] #take only NYC zip codes
data311=pd.pivot_table(data311,index='Zip',columns='Complain',values='Count',fill_value=0)
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,...,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,
10000.0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,8,8,0,0,0,0,0
10001.0,0,1,90,0,0,1,20,36,1,1,...,3,0,2,51,5,5,2,366,0,0
10002.0,15,0,80,0,0,21,18,20,2,6,...,5,1,4,27,14,10,7,324,0,0
10003.0,15,1,143,0,2,44,24,13,2,2,...,7,11,2,54,25,12,5,318,0,0
10004.0,0,0,15,0,0,4,3,1,0,1,...,1,1,0,16,21,0,0,37,0,0


In [48]:
Total311=data311.sum(axis=1) #total 311 activity per zip code
data311=data311.div(data311.sum(axis=1), axis=0) #normalize activity of various cathegories within zip code by total
data311=data311.loc[Total311>100] #keep only those zip codes having sufficient activity

### Question 5
1. Using the Zillow dataset from question 4, run PCA on the price dynamics for zip codes over the last 5 years 
    1. take only those zip codes within NYC having price defined for the last 60 months from 2013-09 till 2018-08:
    2. normalize each zip code timeline by average price over this period
    3. standardize those normalized prices per month and use them as features
    4. run pca over those features

2. Visualize zip codes in the 2d space of first two principal components, coloring them by borough (feel free to use borough definition from the class notebook). 

In [58]:
nyc = zillow.loc[(zillow['Metro'] == 'New York, NY')]
nyc.head()
nyc = nyc.iloc[:,49:]
nyc.head()
nyc = nyc.dropna()
nyc.head(10)

,2013-09,2013-10,2013-11,2013-12,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08
RegionName,,,,,,,,,,,,,,,,,,,,,
10023,1671.539961,1692.579505,1671.539961,1626.483292,1687.241963,1679.678610,1647.985348,1602.992987,1693.398425,1679.748823,...,1954.992968,1983.055556,1963.986600,1702.573836,1522.948540,1568.100358,1612.378790,1569.371728,1599.538839,1619.794484
10016,1208.647968,1240.693215,1338.257652,1288.714051,1341.698842,1287.390942,1290.475408,1361.205586,1357.735068,1343.503937,...,1546.285641,1557.564684,1538.800705,1345.275276,1327.586207,1389.792068,1433.631891,1433.902938,1444.833625,1456.997085
11235,442.216981,442.743009,437.367117,436.607143,436.303571,435.454545,430.821918,435.342870,434.631891,440.149254,...,589.335828,581.314948,578.947368,566.937120,561.356996,582.314205,584.002443,576.515881,577.477570,575.000000
10029,925.000000,946.875000,857.605178,825.504587,857.682409,877.767897,909.326186,1000.175070,940.169668,911.674923,...,1288.980433,1288.980433,1282.051282,1265.954845,1235.999167,1255.656109,1266.935681,1269.503546,1251.968504,1241.534989
10462,149.961919,149.020375,152.517483,153.395836,156.815440,156.001180,156.626733,160.087419,157.142857,157.142857,...,196.202532,200.000000,198.666667,217.787837,220.000000,223.846154,234.939759,228.492107,226.363636,218.090909
11201,993.464052,1014.677031,968.103392,997.150997,1002.227171,1013.942936,1017.699115,1040.264934,1015.371127,1034.768212,...,1362.549358,1374.292643,1362.266858,1314.635408,1131.850675,1211.015262,1275.075415,1284.756100,1285.159067,1296.747967
11214,396.643110,419.029544,422.218623,420.327304,420.327304,420.689968,418.563923,421.052632,435.775452,425.101215,...,593.030508,595.447955,556.794425,531.496063,529.855285,554.545455,589.428794,552.000000,561.936937,601.673642
7030,549.944994,558.922190,563.246281,554.356653,562.379208,576.254146,579.652256,580.004658,578.947368,580.279612,...,732.000000,748.953775,754.175801,748.976808,753.755700,753.196931,752.314815,764.203822,749.343949,746.958378
11229,365.176152,375.583192,395.599490,394.770408,389.398937,331.057988,330.357143,319.901316,320.216049,320.058683,...,615.537849,626.999557,615.753425,613.883064,600.693750,618.823529,630.588235,620.765631,605.823157,608.660786
